In [ ]:
# хочу идти по verified
# 1. строчка прайса
# 2. что верифицировано
# 3. что найдено

In [9]:
from db.verified import report_row_num_un_uuid # (0)
# UN skus for indexing
from db.un import get_en_skus

from indexes.tn import build_tn_indexes, add_to_tn_index
from indexes.skus import build_sku_row_idx

from prices.helpers import load_pulse # (1)
from helpers.skus import get_tn_from_data # ,  get_tn



# from zones.tn import tn_zone
# from zones.company import through_company_zone
# from indexes.company import build_company_idx

# Verified

In [5]:
### price_row_num, un_id
### OrderedDict([(39, 'a4fb182c-9d71-4d62-a405-70947f1dcc4e'),
###             (40, 'a4fb182c-9d71-4d62-a405-70947f1dcc4e'), ...])
verified = report_row_num_un_uuid() # (0)

# Price list

In [6]:
plist = load_pulse() # (1)

# Indexes

1. skus
2. trade_name

In [10]:
%time un_skus = get_en_skus()

sku_indexes = build_sku_row_idx()
# skus_rows_id_idx = sku_indexes['rows_id'] # {..., [`${row_num}`]: un_id, ...}
skus_id_rows_idx = sku_indexes['rows_id_inv'] # {..., [`${un_id}`]: row_num, ...}

%time tn_indexes = build_tn_indexes()

CPU times: user 2.9 s, sys: 347 ms, total: 3.24 s
Wall time: 4.94 s
CPU times: user 57.9 ms, sys: 1.1 ms, total: 59 ms
Wall time: 59.2 ms


In [17]:
def show_sku(sku_rec: tuple):
    sku_id, sku_data = sku_rec
    tn = get_tn_from_data(sku_data)
    lform = get_
#     lform, dosage, count, pack, company = ('конц-т д/приг. р-ра инфуз.', ' 250 мг 5 мл.', 'фл.', 'х1', 'company')
#     msg = '{} {} {} {} {}'.format(tn, lform, dosage, pack, count, company)
    return '!!! {} {}'.format(tn, sku_data)

In [18]:
verified_items = list(verified.items())[:10]
for i, (price_i, un_id) in enumerate(verified_items):
    # price
    sel_plist = plist[price_i - 1:price_i]
    sel_cols = sel_plist[sel_plist.columns[:3]]
    values = sel_cols.values[0]
    code, name, company = values
    print('[{}]\nP: {} [{}]'.format(i, name, company))
    # varified
    # un_id => str
    sku_row = skus_id_rows_idx[un_id]
    un_sku_data = un_skus[sku_row]
    sku_msg = show_sku(un_sku_data)
    print('U: {}'.format(sku_msg))


[0]
P: 5-Нок табл. 50 мг х50 [Lek – Вектор]
U: !!! 5-нок {'packs': {'count': 1, 'children': [{'count': 1, 'count_end': None, 'level_type': 'primary', 'pack_type_id': 'cac499b7-c76f-42d2-86d3-ca876ae92ff5', 'man_form_packs': [{'man_form_key': '0bc4ddded699fe391059188fd67fe38160bef14d014dcae4c3c0ba9dfbb437a2', 'base_measure_unit_id': 'f8d9c546-c4c4-465b-a2ac-398ba9794921', 'base_measure_unit_count': 50, 'base_measure_unit_count_end': None}]}], 'count_end': None, 'level_type': 'consumer', 'pack_type_id': '37d25638-0365-43b8-b87b-f1837a2b30b2'}, 'purpose': None, 'is_recipe': True, 'man_forms': [{'key': '0bc4ddded699fe391059188fd67fe38160bef14d014dcae4c3c0ba9dfbb437a2', 'inns': [{'inn_id': '77e8fd66-69d4-4356-ba5a-744f1d60d2af', 'base_measure_unit_id': '5d4f8663-d75e-4c69-a3fc-c0b8036d896a', 'base_measure_unit_count': 0.05, 'base_measure_unit_count_end': None}], 'dosage_form_id': 'aef3b0ca-d3ef-4d32-a078-923f8c6426ce'}], 'address_id': '6453d7e8-4b4b-4d6a-a4b4-86bb72485678', 'trade_name': '5

In [28]:
get_tn(60249)

'5-нок'

In [38]:
name_bow = get_bow(name)
name_bow

['5-нок', 'табл.', '50', 'мг', 'х50']

In [41]:
term0 = name_bow[0]

In [59]:
def term_docs(term='нок', terms_docs=terms_docs):
    if term not in terms_docs:
        return print('[-]')
    docs = terms_docs[term]
    print(len(docs), len(set(docs)))
    for doc in docs:
        print(get_tn(doc))

In [ ]:
indexes = build_tn_indexes()
# в верефицированных
rows_id_inv = indexes['rows_id_inv']
rows_id_inv['a4fb182c-9d71-4d62-a405-70947f1dcc4e']
terms_docs = indexes['terms_docs']
indexes.keys()

# END TRADE_NAME

[?] как добавить в индекс add_to_tn_index(term='5', docs=[60249])

In [71]:
def add_to_tn_index(term, doc):
    # get existing item or new list
    term_docs = term in terms_docs and terms_docs[term] or list()
    if doc in term_docs:
        return print('there is a doc[{}] in term[{}]'.format(doc, term))
    # can optimizate, where insert?
    term_docs.append(doc)
    newdocs = sorted(term_docs)
    terms_docs[term] = newdocs

# TN RULES

In [72]:
term = '5'
add_to_tn_index(term, doc=60249)
term = 'нок'
add_to_tn_index(term, doc=60249)

there is a doc[60249] in term[5]
there is a doc[60249] in term[нок]


In [60]:
term_docs()

[-]


In [74]:
tn_zone(name, terms_docs)

[[('5-нок', [60249]),
  ('табл.', None),
  ('50',
   [9203,
    24099,
    24104,
    29817,
    50342,
    50343,
    50344,
    91156,
    106090,
    131915,
    131916,
    132005]),
  ('мг', [68457]),
  ('х50', None)]]

# Company

In [127]:
from db.pg import pg_con

def get_addr_company_table():
    conn, cursor = pg_con(dbname='cmpny')
    q = 'select a.id, a.company_id from addresses as a order by a.id;'
    cursor.execute(q)
    recs = cursor.fetchall()
    conn.close()
    conn = None
    cursor = None
    addr = list()
    cmpy = dict()
    addr = [(addr_id, c_id) for addr_id, c_id in recs]
    for addr_id, c_id in recs:
        docs = c_id in cmpy and cmpy[c_id] or list()
        docs.append(addr_id)
        cmpy[c_id] = docs
        
    return dict(recs=recs,
                addr_cmp=dict(addr),
                cmp_addreses=cmpy)

In [128]:
# company_idx = build_company_idx()
# company_terms_docs = company_idx['terms_docs']
# company_id_idx_inv = company_idx['company_id_idx_inv']
# companies_table = company_idx['companies_table']

# un addresses 

addr_company_idx = get_addr_company_table()
addr_cmp_idx = addr_company_idx['addr_cmp'] #
cmp_addr_idx = addr_company_idx['cmp_addreses']
print(addr_company_idx.keys())

company_idx.keys()

dict_keys(['recs', 'addr_cmp', 'cmp_addreses'])


dict_keys(['company_id_idx', 'company_id_idx_inv', 'terms_docs', 'companies_table', 'companies_synonyms'])

In [137]:
company_id_idx = company_idx['company_id_idx']
company_id_idx

In [5]:
# sel_plist_3_fields = sel_plist[sel_plist.columns[:3]].values
# company_zone = company_index_to_docs(through_company_zone(sel_plist_3_fields, company_terms_docs))

In [95]:
def company_index_to_docs(res):
    print(res)

In [ ]:
# row_num -> company_id
# TODO: row_num -> docs
company_row = 2696
company_id = company_id_idx_inv[company_row]
print('company_id {}'.format(company_id))
addresses_cmp_id = cmp_addr_idx[addr_cmp_idx['64cb8085-926e-414a-90cf-d38a1dc4ef0b']]
print('company_id address list: {}'.format(addresses_cmp_id))
print('TODO: list of skus:')
# print('company_id {}'.format(company_id))
# # print(companies_table[company_row])
# cmp_addr_idx